Exploring for Potential New Hotel Venues in Toronto

First, we start by importing the required libraries.

In [77]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
from sklearn.datasets.samples_generator import make_blobs
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
from bs4 import BeautifulSoup
import lxml
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [9]:
my_url='https://en.wikipedia.org/wiki/Demographics_of_Toronto_neighbourhoods'

In [10]:
r=requests.get(my_url)

In [11]:
# Parse the htlm with Soup
page=BeautifulSoup(r.text,"html.parser")
page


<!DOCTYPE html>

<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>Demographics of Toronto neighbourhoods - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"XoAlTQpAAL0AAH076YsAAAAU","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Demographics_of_Toronto_neighbourhoods","wgTitle":"Demographics of Toronto neighbourhoods","wgCurRevisionId":947857603,"wgRevisionId":947857603,"wgArticleId":38958143,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with obsolete information from April 2013","All Wikipedia articles in need of updating","Neigh

In [12]:
response_obj = requests.get('https://en.wikipedia.org/wiki/Demographics_of_Toronto_neighbourhoods').text
soup = BeautifulSoup(response_obj,'lxml')
Neighbourhoods_Toronto = soup.find('table',{'class':'wikitable'})
Neighbourhoods_Toronto

<table border="1" cellpadding="5" cellspacing="0" class="wikitable sortable">
<tbody><tr>
<th width="20%">Name
</th>
<th width="5%">FM
</th>
<th width="20%">Census Tracts
</th>
<th width="5%">Population
</th>
<th width="5%">Land area (km2)
</th>
<th width="5%">Density (people/km2)
</th>
<th width="5%">% Change in Population since 2001
</th>
<th width="5%">Average Income
</th>
<th width="5%">Transit Commuting %
</th>
<th width="5%">% Renters
</th>
<th width="10%">Second most common language (after English) by name
</th>
<th width="10%">Second most common language (after English) by percentage
</th>
<th width="10%">Map
</th></tr>
<tr>
<td><b>Toronto <a class="mw-redirect" href="/wiki/Census_metropolitan_area" title="Census metropolitan area">CMA</a> Average</b>
</td>
<td>
</td>
<td>All
</td>
<td><b>5,113,149</b>
</td>
<td><b>5903.63</b>
</td>
<td><b>866</b>
</td>
<td><b>9.0</b>
</td>
<td><b>40,704</b>
</td>
<td><b>10.6</b>
</td>
<td><b>11.4</b>
</td>
<td>
</td>
<td>
</td>
<td>
</td></tr>

In [13]:
totals=Neighbourhoods_Toronto.find_all('tr')
nrows=len(totals)
nrows

176

The Header of the DataFrame is the first row of data

In [14]:
header=totals[0].text.split()
header

['Name',
 'FM',
 'Census',
 'Tracts',
 'Population',
 'Land',
 'area',
 '(km2)',
 'Density',
 '(people/km2)',
 '%',
 'Change',
 'in',
 'Population',
 'since',
 '2001',
 'Average',
 'Income',
 'Transit',
 'Commuting',
 '%',
 '%',
 'Renters',
 'Second',
 'most',
 'common',
 'language',
 '(after',
 'English)',
 'by',
 'name',
 'Second',
 'most',
 'common',
 'language',
 '(after',
 'English)',
 'by',
 'percentage',
 'Map']

Checking row 3 as sample

In [15]:
totals[3].text

'\nAlderwood\n\nE\n\n0211.00, 0212.00\n\n11,656\n\n4.94\n\n2360\n\n-4.0\n\n35,239\n\n8.8\n\n8.5\n\nPolish (6.2%)\n\n06.2% Polish\n\n\n'

In [16]:
totals[3].text.split('\n')

['',
 'Alderwood',
 '',
 'E',
 '',
 '0211.00, 0212.00',
 '',
 '11,656',
 '',
 '4.94',
 '',
 '2360',
 '',
 '-4.0',
 '',
 '35,239',
 '',
 '8.8',
 '',
 '8.5',
 '',
 'Polish (6.2%)',
 '',
 '06.2% Polish',
 '',
 '',
 '']

In [17]:
Neighbourhood=totals[3].text.split('\n')[1]
Neighbourhood

'Alderwood'

Extracting cells into a dataframe

In [18]:
records =[]
n=1
while n < nrows :
    Neighbourhood=totals[n].text.split('\n')[1]
    Size=totals[n].text.split('\n')[9]
    Population=totals[n].text.split('\n')[7]
    records.append((Neighbourhood,Size,Population))
    n=n+1

df1=pd.DataFrame(records, columns=['Neighbourhood', 'Land Area', 'Population'])
df1.head(11)

,Neighbourhood,Land Area,Population
0,Toronto CMA Average,5903.63,"5,113,149"
1,Agincourt,12.45,"44,577"
2,Alderwood,4.94,"11,656"
3,Alexandra Park,0.32,"4,355"
4,Allenby,0.58,"2,513"
5,Amesbury,3.51,"17,318"
6,Armour Heights,2.29,"4,384"
7,Banbury,2.72,"6,641"
8,Bathurst Manor,4.69,"14,945"
9,Bay Street Corridor,0.11,"4,787"


In [19]:
# download postal codes data and parse it:
source = requests.get("https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=942851379").text
soup=BeautifulSoup(source,'lxml')
print(soup.title)
from IPython.display import display_html
tab = str(soup.table)
display_html(tab,raw=True)

<title>List of postal codes of Canada: M - Wikipedia</title>


Postcode,Borough,Neighbourhood
M1A,Not assigned,Not assigned
M2A,Not assigned,Not assigned
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,Harbourfront
M6A,North York,Lawrence Heights
M6A,North York,Lawrence Manor
M7A,Downtown Toronto,Queen's Park
M8A,Not assigned,Not assigned
M9A,Etobicoke,Islington Avenue


In [20]:
dfs = pd.read_html(tab)
df=dfs[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [21]:
df = df[~df['Borough'].isnull()]  # to filter out bad rows
df.drop(df[df.Borough == 'Not assigned'].index, inplace=True)
df.reset_index(drop=True, inplace=True)
df = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda x: ','.join(x)).reset_index()
df['Neighbourhood'].replace('Not assigned',df['Borough'],inplace=True)
df

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


Calculating Latitude and Longitude per Neighbourhood

In [24]:
df_geo = pd.read_csv('http://cocl.us/Geospatial_data')
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merging data into a table

In [22]:
# merge
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [25]:
df_geo.reset_index()
df_geo.columns = ['Postcode','Latitude','Longitude']

#joining two dataframes on the Postcode column 
df_new = pd.merge(df,df_geo,on = 'Postcode')
df_new.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


Merging data with population data

In [28]:
df2s=df1.sort_values('Neighbourhood')

In [29]:
Neighbourhoods=pd.merge(df2s,df_new, how='right', on = 'Neighbourhood')
Neighbourhoods.head(12)

,Neighbourhood,Land Area,Population,Postcode,Borough,Latitude,Longitude
0,Agincourt,12.45,"44,577",M1S,Scarborough,43.794200,-79.262029
1,Bayview Village,4.14,"12,280",M2K,North York,43.786947,-79.385975
2,Church and Wellesley,0.55,"13,397",M4Y,Downtown Toronto,43.665860,-79.383160
3,Davisville,3.14,"23,727",M4S,Central Toronto,43.704324,-79.388790
4,Humber Summit,7.89,"12,766",M9L,North York,43.756303,-79.565963
5,Lawrence Park,3.64,"6,653",M4N,Central Toronto,43.728020,-79.388790
6,Leaside,2.81,"13,876",M4G,East York,43.709060,-79.363452
7,Parkwoods,4.96,"26,533",M3A,North York,43.753259,-79.329656
8,Rosedale,2.72,"7,672",M4W,Downtown Toronto,43.679563,-79.377529
9,Scarborough Village,2.03,"12,796",M1J,Scarborough,43.744734,-79.239476


In [31]:
Neighbourhoods.shape

(103, 7)

Hence, there are 103 rows after merging the dataframe with the geographical data, with 7 columns after adding latitude and longitude

Clustering Toronto's Neighbourhoods

First, we will connect to  Foursquare to explore and segment the data

In [40]:
# Define Foursquare Credentials and Version
CLIENT_ID = '5KVG0ZSJYTPJTXRMUCBZOWNFPZFXAYI1GXWT2331E0DV4DSK' # your Foursquare ID
CLIENT_SECRET = '4V145UIFLD3CIS5XUS3DLCB4T031TDUXLPGNUVVQG1RYT0YC' # your Foursquare Secret
VERSION = '20180604'

In [41]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 2000 # define radius

Now, we are going to map Toronto

In [42]:
# get the geographical coordinates of Downtown Toronto
address = 'Downtown Toronto, ON, Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude_downtown_toronto = location.latitude
longitude_downtown_toronto = location.longitude
print("Downtown Toronto","latitude",latitude_downtown_toronto, "& " "longitude" ,longitude_downtown_toronto)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.


Downtown Toronto latitude 43.6563221 & longitude -79.3809161


In [80]:
# create map of Downtown Toronto using latitude and longitude values
map_downtown_toronto = folium.Map(location=[latitude_downtown_toronto,longitude_downtown_toronto], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_new['Latitude'], df_new['Longitude'], df_new['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown_toronto)  
    
map_downtown_toronto

In [44]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category' ]
    
    return(nearby_venues)

Next, we will write the code to run the function on each neighborhood and create a new dataframe called Toronto_venues

In [47]:
Toronto_venues = getNearbyVenues(names=Neighbourhoods['Neighbourhood'],
                                   latitudes=Neighbourhoods['Latitude'],
                                   longitudes=Neighbourhoods['Longitude']
                                    )

Agincourt
Bayview Village
Church and Wellesley
Davisville
Humber Summit
Lawrence Park
Leaside
Parkwoods
Rosedale
Scarborough Village
St. James Town
The Beaches
Thorncliffe Park
Victoria Village
Westmount
Weston
Woburn
Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Cedarbrae
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview,Henry Farm,Oriole
Silver Hills,York Mills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Downsview North,Wilson Heights
Northwood Park,York University
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Woodbine 

In [48]:
print(Toronto_venues.shape)
Toronto_venues.head(15)

(2197, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Agincourt,43.794200,-79.262029,Panagio's Breakfast & Lunch,43.792370,-79.260203,Breakfast Spot
1,Agincourt,43.794200,-79.262029,El Pulgarcito,43.792648,-79.259208,Latin American Restaurant
2,Agincourt,43.794200,-79.262029,Twilight,43.791999,-79.258584,Lounge
3,Agincourt,43.794200,-79.262029,Mark's,43.791179,-79.259714,Clothing Store
4,Agincourt,43.794200,-79.262029,Commander Arena,43.794867,-79.267989,Skating Rink
5,Bayview Village,43.786947,-79.385975,Sun Star Chinese Cuisine 翠景小炒,43.787914,-79.381234,Chinese Restaurant
6,Bayview Village,43.786947,-79.385975,TD Canada Trust,43.788074,-79.380367,Bank
7,Bayview Village,43.786947,-79.385975,Maxim's Cafe and Patisserie,43.787863,-79.380751,Café
8,Bayview Village,43.786947,-79.385975,Kaga Sushi,43.787758,-79.381090,Japanese Restaurant
9,Church and Wellesley,43.665860,-79.383160,Storm Crow Manor,43.666840,-79.381593,Theme Restaurant


Number of venues per Neighbourhood

In [49]:
Toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Agincourt,5,5,5,5,5,5
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",2,2,2,2,2,2
"Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",9,9,9,9,9,9
"Alderwood,Long Branch",8,8,8,8,8,8
"Bathurst Manor,Downsview North,Wilson Heights",20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4
"Bedford Park,Lawrence Manor East",24,24,24,24,24,24
Berczy Park,56,56,56,56,56,56


Number of unique categories

In [50]:
print('The number of unique categories is {}.'.format(len(Toronto_venues['Venue Category'].unique())))

The number of unique categories is 270.


Now, we will analyze the Neighbourhoods in Toronto to understand what are the top 5 most common venues 

In [52]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighbourhood'] = Toronto_venues['Neighbourhood'] 

# move district column to the first column
cols=list(Toronto_onehot.columns.values)
cols.pop(cols.index('Neighbourhood'))
Toronto_onehot=Toronto_onehot[['Neighbourhood']+cols]

Toronto_onehot.rename(columns = {'Neighbourhood': 'Neighbourhood'}, inplace = True)
Toronto_onehot.head(15)

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Social Club,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [53]:
Toronto_onehot.shape

(2197, 271)

Grouping rows by Neighbourhood and by the mean of the frequency of occurrence of each category

In [54]:
Toronto_grouped = Toronto_onehot.groupby('Neighbourhood').mean().reset_index()
Toronto_grouped

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Social Club,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide,King,Richmond",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.020000,0.000000,0.00,0.010000,0.000000,0.020

In [55]:
Toronto_grouped.shape

(98, 271)

Printing Neighbourhoods along with the top 5 most common venues

In [56]:
num_top_venues = 5

for hood in Toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
         venue  freq
0  Coffee Shop  0.07
1   Restaurant  0.06
2         Café  0.05
3          Bar  0.03
4       Bakery  0.03


----Agincourt----
                       venue  freq
0             Breakfast Spot   0.2
1                     Lounge   0.2
2             Clothing Store   0.2
3               Skating Rink   0.2
4  Latin American Restaurant   0.2


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
                       venue  freq
0                 Playground   0.5
1                       Park   0.5
2          Accessories Store   0.0
3  Middle Eastern Restaurant   0.0
4                      Motel   0.0


----Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
                 venue  freq
0        Grocery Store  0.22
1          Pizza Place  0.11
2         Liquor Store  0.11
3       Sandwich Place  0.11
4  Fried Chicken Joint  0.11


----Alderwood,Long Branch----
            venue  f

In [57]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [58]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = Toronto_grouped['Neighbourhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)
neighbourhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Restaurant,Café,Bar,Bakery,Clothing Store,Office,Pizza Place,Lounge,Gym
1,Agincourt,Clothing Store,Lounge,Breakfast Spot,Skating Rink,Latin American Restaurant,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Park,Playground,Yoga Studio,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Pharmacy,Fried Chicken Joint,Pizza Place,Sandwich Place,Liquor Store,Beer Store,Fast Food Restaurant,German Restaurant,Department Store
4,"Alderwood,Long Branch",Pizza Place,Pharmacy,Gym,Skating Rink,Coffee Shop,Pub,Sandwich Place,Dog Run,Diner,Discount Store
5,"Bathurst Manor,Downsview North,Wilson Heights",Coffee Shop,Bank,Ice Cream Shop,Restaurant,Chinese Restaurant,Shopping Mall,Deli / Bodega,Pizza Place,Gas Station,Supermarket
6,Bayview Village,Café,Bank,Chinese Restaurant,Japanese Restaurant,Yoga Studio,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
7,"Bedford Park,Lawrence Manor East",Coffee Shop,Sandwich Place,Italian Restaurant,Restaurant,Pizza Place,Indian Restaurant,Sushi Restaurant,Café,Juice Bar,Thai Restaurant
8,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Restaurant,Café,Seafood Restaurant,Cheese Shop,Farmers Market,Beer Bar,Shopping Mall
9,"Birch Cliff,Cliffside West",College Stadium,Café,Skating Rink,General Entertainment,Donut Shop,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore


Clustering Neighbourhoods

In [87]:
kmeans.labels_.shape

(98,)

In [92]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

Toronto_merged = neighbourhoods_venues_sorted

Toronto_merged['Cluster Labels'] = kmeans.labels_

Toronto_merged

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,"Adelaide,King,Richmond",Coffee Shop,Restaurant,Café,Bar,Bakery,Clothing Store,Office,Pizza Place,Lounge,Gym,1
1,Agincourt,Clothing Store,Lounge,Breakfast Spot,Skating Rink,Latin American Restaurant,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,1
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Park,Playground,Yoga Studio,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,2
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Pharmacy,Fried Chicken Joint,Pizza Place,Sandwich Place,Liquor Store,Beer Store,Fast Food Restaurant,German Restaurant,Department Store,1
4,"Alderwood,Long Branch",Pizza Place,Pharmacy,Gym,Skating Rink,Coffee Shop,Pub,Sandwich Place,Dog Run,Diner,Discount Store,1
5,"Bathurst Manor,Downsview North,Wilson Heights",Coffee Shop,Bank,Ice Cream Shop,Restaurant,Chinese Restaurant,Shopping Mall,Deli / Bodega,Pizza Place,Gas Station,Supermarket,1
6,Bayview Village,Café,Bank,Chinese Restaurant,Japanese Restaurant,Yoga Studio,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,1
7,"Bedford Park,Lawrence Manor East",Coffee Shop,Sandwich Place,Italian Restaurant,Restaurant,Pizza Place,Indian Restaurant,Sushi Restaurant,Café,Juice Bar,Thai Restaurant,1
8,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Restaurant,Café,Seafood Restaurant,Cheese Shop,Farmers Market,Beer Bar,Shopping Mall,1
9,"Birch Cliff,Cliffside West",College Stadium,Café,Skating Rink,General Entertainment,Donut Shop,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore,1


In [101]:
# merge Toronto_grouped with Toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = neighbourhoods_venues_sorted.merge(Neighbourhoods, on='Neighbourhood', how='left')

Toronto_merged.head(11)

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels,Land Area,Population,Postcode,Borough,Latitude,Longitude
0,"Adelaide,King,Richmond",Coffee Shop,Restaurant,Café,Bar,Bakery,Clothing Store,Office,Pizza Place,Lounge,Gym,1,NaN,NaN,M5H,Downtown Toronto,43.650571,-79.384568
1,Agincourt,Clothing Store,Lounge,Breakfast Spot,Skating Rink,Latin American Restaurant,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,1,12.45,"44,577",M1S,Scarborough,43.794200,-79.262029
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Park,Playground,Yoga Studio,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,2,NaN,NaN,M1V,Scarborough,43.815252,-79.284577
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Pharmacy,Fried Chicken Joint,Pizza Place,Sandwich Place,Liquor Store,Beer Store,Fast Food Restaurant,German Restaurant,Department Store,1,NaN,NaN,M9V,Etobicoke,43.739416,-79.588437
4,"Alderwood,Long Branch",Pizza Place,Pharmacy,Gym,Skating Rink,Coffee Shop,Pub,Sandwich Place,Dog Run,Diner,Discount Store,1,NaN,NaN,M8W,Etobicoke,43.602414,-79.543484
5,"Bathurst Manor,Downsview North,Wilson Heights",Coffee Shop,Bank,Ice Cream Shop,Restaurant,Chinese Restaurant,Shopping Mall,Deli / Bodega,Pizza Place,Gas Station,Supermarket,1,NaN,NaN,M3H,North York,43.754328,-79.442259
6,Bayview Village,Café,Bank,Chinese Restaurant,Japanese Restaurant,Yoga Studio,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,1,4.14,"12,280",M2K,North York,43.786947,-79.385975
7,"Bedford Park,Lawrence Manor East",Coffee Shop,Sandwich Place,Italian Restaurant,Restaurant,Pizza Place,Indian Restaurant,Sushi Restaurant,Café,Juice Bar,Thai Restaurant,1,NaN,NaN,M5M,North York,43.733283,-79.419750
8,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Restaurant,Café,Seafood Restaurant,Cheese Shop,Farmers Market,Beer Bar,Shopping Mall,1,NaN,NaN,M5E,Downtown Toronto,43.644771,-79.373306
9,"Birch Cliff,Cliffside West",College Stadium,Café,Skating Rink,General Entertainment,Donut Shop,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore,1,NaN,NaN,M1N,Scarborough,43.692657,-79.264848


In [102]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[1:10]



array([1, 2, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

Clusters visualization of Toronto's Neighbourhoods

In [103]:
# create map
map_clusters = folium.Map(location=[latitude_downtown_toronto,longitude_downtown_toronto], zoom_start=11)

In [104]:
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

In [111]:
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighbourhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Cluster 0 - Most common venues: Restaurants, Dung Run, and Yoga Studio (No Hotels in Top 10 venues)

In [106]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[0] + list(range(5, Toronto_merged.shape[1]))]]

,Neighbourhood,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels,Land Area,Population,Postcode,Borough,Latitude,Longitude
17,Canada Post Gateway Processing Centre,Fried Chicken Joint,Mediterranean Restaurant,Burrito Place,American Restaurant,Sandwich Place,Dog Run,0,NaN,NaN,M7R,Mississauga,43.636966,-79.615819
25,"Cliffcrest,Cliffside,Scarborough Village West",Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio,Diner,0,NaN,NaN,M1M,Scarborough,43.716316,-79.239476
90,Victoria Village,Yoga Studio,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,0,4.72,"17,047",M4A,North York,43.725882,-79.315572
94,Woburn,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,0,13.34,"48,507",M1G,Scarborough,43.770992,-79.216917


Cluster 1 - Most common venues: Restaurants, Nightlife, Gas Stations, Cafes, Gym/Sports and Hotels

In [107]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[0] + list(range(5, Toronto_merged.shape[1]))]]

,Neighbourhood,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels,Land Area,Population,Postcode,Borough,Latitude,Longitude
0,"Adelaide,King,Richmond",Bakery,Clothing Store,Office,Pizza Place,Lounge,Gym,1,NaN,NaN,M5H,Downtown Toronto,43.650571,-79.384568
1,Agincourt,Latin American Restaurant,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,1,12.45,"44,577",M1S,Scarborough,43.794200,-79.262029
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Sandwich Place,Liquor Store,Beer Store,Fast Food Restaurant,German Restaurant,Department Store,1,NaN,NaN,M9V,Etobicoke,43.739416,-79.588437
4,"Alderwood,Long Branch",Coffee Shop,Pub,Sandwich Place,Dog Run,Diner,Discount Store,1,NaN,NaN,M8W,Etobicoke,43.602414,-79.543484
5,"Bathurst Manor,Downsview North,Wilson Heights",Chinese Restaurant,Shopping Mall,Deli / Bodega,Pizza Place,Gas Station,Supermarket,1,NaN,NaN,M3H,North York,43.754328,-79.442259
6,Bayview Village,Yoga Studio,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,1,4.14,"12,280",M2K,North York,43.786947,-79.385975
7,"Bedford Park,Lawrence Manor East",Pizza Place,Indian Restaurant,Sushi Restaurant,Café,Juice Bar,Thai Restaurant,1,NaN,NaN,M5M,North York,43.733283,-79.419750
8,Berczy Park,Café,Seafood Restaurant,Cheese Shop,Farmers Market,Beer Bar,Shopping Mall,1,NaN,NaN,M5E,Downtown Toronto,43.644771,-79.373306
9,"Birch Cliff,Cliffside West",Donut Shop,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore,1,NaN,NaN,M1N,Scarborough,43.692657,-79.264848
10,"Bloordale Gardens,Eringate,Markland Wood,Old B...",Café,Pizza Place,Liquor Store,Event Space,Ethiopian Restaurant,Falafel Restaurant,1,NaN,NaN,M9C,Etobicoke,43.643515,-79.577201


 Cluster 2 - Most common venues: Restaurants, Discount Stores, Dog Run , Distrubition Centers (No Hotels in Top 10 venues)

In [108]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[0] + list(range(5, Toronto_merged.shape[1]))]]

,Neighbourhood,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels,Land Area,Population,Postcode,Borough,Latitude,Longitude
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,2,NaN,NaN,M1V,Scarborough,43.815252,-79.284577
13,"CFB Toronto,Downsview East",Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,2,NaN,NaN,M3K,North York,43.737473,-79.464763
16,Caledonia-Fairbanks,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Electronics Store,2,NaN,NaN,M6E,York,43.689026,-79.453512
40,East Toronto,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,2,NaN,NaN,M4J,East York,43.685347,-79.338106
71,Parkwoods,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,2,4.96,"26,533",M3A,North York,43.753259,-79.329656
73,Rosedale,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,2,2.72,"7,672",M4W,Downtown Toronto,43.679563,-79.377529
88,"The Kingsway,Montgomery Road,Old Mill North",Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,2,NaN,NaN,M8X,Etobicoke,43.653654,-79.506944
97,York Mills West,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,2,NaN,NaN,M2P,North York,43.752758,-79.400049


 Cluster 3 - Most common venues:  Restaurants, Distribution Centers, and Dog Run (No Hotels in Top 10 venues)

In [109]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[0] + list(range(5, Toronto_merged.shape[1]))]]

,Neighbourhood,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels,Land Area,Population,Postcode,Borough,Latitude,Longitude
68,Northwest,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Dim Sum Restaurant,3,NaN,NaN,M9W,Etobicoke,43.706748,-79.594054


 Cluster 4 - Most common venues: Restaurants, Drugstore, Dog Run (No Hotels in Top 10 venues)

In [110]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[0] + list(range(5, Toronto_merged.shape[1]))]]

,Neighbourhood,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels,Land Area,Population,Postcode,Borough,Latitude,Longitude
41,"Emery,Humberlea",Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Field,4,NaN,NaN,M9M,North York,43.724766,-79.532242


Results
The following are the highlights of the 5 clusters above:
1. Hotels are clearly located only in Cluster 1, which makes the selection of the final location for a new hotel easily identifiable in order for company ABC to reduce risks
2. Restaurants, nightlife, and coffe shops are very popular in Cluster 1 as well
3. As for Yoga, this is clearly allocated in Cluster 0 
4. Although, the Clusters have variations, a very visible presence is the predominance of bars and restaurants, which is typical for any big city city

Discussion and Conclusion
It is noticable that Cluster 1 is the most viable cluster to build a new luxury hotel. The proximity to a big number of Restaurants (lunch and dinner venues for guests), Coffee shops and other amenities are also very important points to take into account when making the right choice. This cluster Hotels in the top venues in Central Toronto, Downtown Toronto and North York.
 In conclusion, this project would have had better results if there were more available data in terms of actual land pricing data within the area, public transportantion access and allowance of more venues exploration with the Foursquare (limited venues for free calls).
However, based on the available data, the analysis shows that the best option for Company ABC would be to focus on only three Toronto Neighbourhoods (Central Toronto, Downtown Toronto and North York) when establishing a new hotel.
